# SPECFEM Users Workshop (Oct. 5-7, 2022)
-------------------------

## Container Testing Instructions

- Day 0 instructions are meant for Users to test out their Docker container to ensure that things will go smoothly during the workshop.  
- Estimated times are provided next to each task to provide a benchmark on how fast thing should run. Benchmarks were run on a 2015 Macbook Pro (OSX 10.14.6) with 3.1GHz Intel Core i7, and 16Gb of ram. 
- If you run into any issues while running through this notebook, please copy the error message and make note of which cell created the error.
- These instructions should be run from inside the Docker container, using Jupyter Lab. The Docker container should have the adjTomo toolkit installed (SeisFlows, Pyatoa, PySEP), as well as SPECFEM2D and SPECEFM3D compiled with MPI. 

--------------------

**Jupyter Quick Tips:**

- **Run cells** one-by-one by hitting the $\blacktriangleright$ button at the top, or by hitting `Shift + Enter`
- **Run all cells** by hitting the $\blacktriangleright\blacktriangleright$ button at the top, or by running `Run -> Run All Cells`
- **Currently running cells** that are still processing will have a `[*]` symbol next to them
- **Finished cells** will have a `[1]` symbol next to them. The number inside the brackets represents what order this cell has been run in.
- Commands that start with `!` are Bash commands (i.e., commands you would run from the terminal)
- Commands that start with `%` are Jupyter Magic commands.
- To time a task, put a `%time` before the command (e.g., `%time ! ls`)


**Relevant Links:** 
- Workshop announcement: https://sites.google.com/alaska.edu/carltape/home/research/specfem2022?authuser=0
- adjTomo Container: https://github.com/SeisSCOPED/adjtomo
- adjTomo Org Page: https://github.com/adjTomo
- SeisFlows docs: https://seisflows.readthedocs.io/en/devel/
- Pyatoa docs: https://pyatoa.readthedocs.io/en/devel/
- PySEP readme: https://github.com/uafgeotools/pysep#readme

----------------

**Docker Preamble:**  

To open this container you will need the latest container.   
**> Please run the following from your Terminal**

```bash
docker pull ghcr.io/seisscoped/adjtomo:ubuntu20.04
```

To save the results we obtain from inside our container, we will need to mount our local filesystem.  
**> Please `cd` (change directory) to am empty working directory before running the following command:**

```bash
docker run -p 8888:8888 --mount type=bind,source=$(pwd),target=/home/scoped/work --shm-size=1gb ghcr.io/seisscoped/adjtomo:ubuntu20.04 
```

>__Docker command explanation__:
The `pull` command downloads the Docker Image from GitHub. The Docker `run` command launches the Docker container. The `--mount` flag binds our local filesystem (in the current working directory, pwd) with the container's internal filesytem (at location /home/scoped/work). The `--shm-size` flag  tells Docker to give us 512 Megabytes of shared memory, which is required for MPI processes.

>__SIGBUS Error__: If you are receiving the error message *Program received signal SIGBUS: Access to an undefined portion of a memory object.*, please `File -> Shut Down` the current container and reopen with a larger value for the `--shm-size` flag above. e.g., `--shm-size=2gb`. 


## 1) Testing SPECFEM2D

We want to make sure we can run SPECFEM2D natively by running the example problem using MPI.  
**Benchmark time for cell w/ 4 MPI processes: ~60s**

In [ ]:
%cd /home/scoped/specfem2d

# Convenience function to edit the SPECFEM Par_file from the command line
! seisflows sempar -P DATA/Par_file nproc 4  # change number of processors from 1 to 4

! mpirun -n 4 bin/xmeshfem2D
! mpirun -n 4 bin/xspecfem2D

Using the navigation menu on the left, navigate to:

`home -> specfem2d -> OUTPUT_FILES`

**Open** any of the `forward_image*.jpg` files and see that the wavefront makes sense. It should look like a layered halfspace with topography and a cavity in the second layer.

## 2) SeisFlows Setup

Some directory bookkeeping to make sure we keep the container `home` directory free of clutter

In [ ]:
%cd /home/scoped/work  
! mkdir example_1 example_2 example_2a example_3 

## 3) Example 1: Homogeneous Halfspace Inversion

This example runs a 1 iteration inversion for a single source-receiver pair, using two homogeneous halfspace models. See [SeisFlows Example \#1 docs page](https://seisflows.readthedocs.io/en/devel/specfem2d_example.html#example-1-homogenous-halfspace-inversion) for guidance on what is going on  
**Benchmark time for cell w/ 2 MPI processes: ~1m45s**

In [10]:
# (optional) Print help dialogue
! seisflows examples 1

No existing SPECFEM2D repo given, default to: /home/scoped/work/example_2/specfem2d

                                    @@@@@@@@@@                        
                               .@@@@.    .%&(  %@.          
                            @@@@   @@@@   &@@@@@@ ,%@       
                         @@@@   @@@,  /@@              @    
                        @@@   @@@@   @@@              @     
                      @@@@   @@@@   @@@                @  @ 
                      @@@   @@@@   ,@@@                @ @  
                     @@@@   @@@@    @@@@              @@ @ @
                     @@@@   @@@@@    @@@@@          @@@ @@ @
                     @@@@    @@@@@     @@@@@@@@@@@@@@  @@  @
                      @@@@    @@@@@@        @@@&     @@@  @ 
                      @@@@@     @@@@@@@@         %@@@@#  @@ 
                        @@@@#      @@@@@@@@@@@@@@@@@   @@   
                         &@@@@@          @@@@(       @@&    
                            @@@@@@@             /@@

In [ ]:
# Run Example 1 with 2 processors
%cd /home/scoped/work/example_1
%time ! seisflows examples run 1 -r /home/scoped/specfem2d --with_mpi --nproc 2

In [ ]:
# Plots the initial and final models, as well as the gradient 
! seisflows plot2d MODEL_INIT vs --savefig m_init_vs.png
! seisflows plot2d MODEL_TRUE vs --savefig m_true_vs.png
! seisflows plot2d MODEL_01 vs --savefig m_01_vs.png
! seisflows plot2d GRADIENT_01 vs_kernel --savefig g_01_vs.png

Using the navigation menu on the left, navigate to:

`home -> work -> example_1`

and **open** each of the .png files that were created to look at the results of this simple inversion. They should match the figures shown in the documentation page.

## 4) Example 2: Checkerboard inversion 

This example runs a muti-event, multi-station, inversion to receover a target checkerboard model, starting from a homoegeneous halfspace model. See [SeisFlows Example \#2 docs page](https://seisflows.readthedocs.io/en/devel/specfem2d_example.html#example-2-checkerboard-inversion-w-pyaflowa-l-bfgs) for guidance on what is going on. We have slimmed down the total Example size to reduce overall compute time.  
**Benchmark time for cell: ~4m**

In [11]:
# (optional) Print help dialogue
! seisflows examples 2 --niter 1 --ntask 2 --nsta 4

No existing SPECFEM2D repo given, default to: /home/scoped/work/example_2/specfem2d

                                    @@@@@@@@@@                        
                               .@@@@.    .%&(  %@.          
                            @@@@   @@@@   &@@@@@@ ,%@       
                         @@@@   @@@,  /@@              @    
                        @@@   @@@@   @@@              @     
                      @@@@   @@@@   @@@                @  @ 
                      @@@   @@@@   ,@@@                @ @  
                     @@@@   @@@@    @@@@              @@ @ @
                     @@@@   @@@@@    @@@@@          @@@ @@ @
                     @@@@    @@@@@     @@@@@@@@@@@@@@  @@  @
                      @@@@    @@@@@@        @@@&     @@@  @ 
                      @@@@@     @@@@@@@@         %@@@@#  @@ 
                        @@@@#      @@@@@@@@@@@@@@@@@   @@   
                         &@@@@@          @@@@(       @@&    
                            @@@@@@@             /@@

In [ ]:
# Run Example 2 with 1 processor
%cd /home/scoped/work/example_2
! seisflows examples run 2 -r /home/scoped/specfem2d --with_mpi --niter 1 --ntask 2 --nsta 4

In [ ]:
# Plots the target and final models, as well as the gradient 
! seisflows plot2d MODEL_TRUE vs --savefig m_true_vs.png
! seisflows plot2d MODEL_02 vs --savefig m_02_vs.png
! seisflows plot2d GRADIENT_02 vs_kernel --savefig g_02_vs.png

Using the navigation menu on the left, navigate to:

`home -> work -> example_2`

and **open** the .png files that were created to look at the results of this simple inversion. They should match the figures shown in the [documentation page](https://seisflows.readthedocs.io/en/devel/specfem2d_example.html#example-2-checkerboard-inversion-w-pyaflowa-l-bfgs).

## 5) Example 2a: Re-create a Kernel from Tape et al. 2007

This Example runs Example 2, but for only 1 event, in order to generate an event kernel. See [SeisFlows Example \#2a docs page](https://seisflows.readthedocs.io/en/devel/specfem2d_example.html#re-creating-kernels-from-tape-et-al-2007) for guidance on what is going on.  
**Benchmark time for cell: ~4m**

>__NOTE:__
You can choose which event ID you're running by changing the integer after `--event_id`. Tape et al. show results for Event IDs 1 through 7.

In [ ]:
# Run Example 2 for only 1 iteration, and for a given event ID
%cd /home/scoped/work/example_2a
! seisflows examples run 2 -r /home/scoped/specfem2d --with_mpi --niter 1 --event_id 7

In [ ]:
# Plots the event kernel
! seisflows plot2d GRADIENT_01 vs_kernel --savefig g_01_vs.png

Using the navigation menu on the left, navigate to:

`home -> work -> example_2a`

and **open** the .png file that was created. Make sure that the kernel looks like one of the panels provided in [Figure 9 of Tape et al.](https://seisflows.readthedocs.io/en/devel/specfem2d_example.html#re-creating-kernels-from-tape-et-al-2007)

----------

We can also look at waveform figures generated by `Pyatoa`, a misfit quantification package used by SeisFlows to determine data-synthetic misfit. These can be found be navigating to

`home -> work -> example_2 -> output -> pyaflowa -> figures`

There is one .pdf file in there, corresponding to waveforms generated during the first iteration (i01) and first step count (s01). If you open the PDF you'll see waveform comparison figures showing time shifts between data and synthetics.

-------------

If you want to **re-run this example** to generate a new event kernel, you will have to clear out the existing working directory. This is because SeisFlows has a checkpointing feature, which will not allow you to run another workflow within an existing workflow directory.

In [ ]:
! rm -r /home/scoped/work/example_2a
! mkdir /home/scoped/work/example_2a
! seisflows examples run 2 -r /home/scoped/specfem2d --with_mpi --niter 1 --event_id 5

## 6) Example 3: En-masse Forward Simulations

This Example runs forward simulations for 250 source-receiver pairs. See [SeisFlows Example \#3 docs page](https://seisflows.readthedocs.io/en/devel/specfem2d_example.html#example-3-en-masse-forward-simulations) for guidance on what is going on.  
**Benchmark time for cell: ~4m**

In [13]:
# (optional) Print help dialogue
! seisflows examples 3

No existing SPECFEM2D repo given, default to: /home/scoped/work/example_2a/specfem2d

                                    @@@@@@@@@@                        
                               .@@@@.    .%&(  %@.          
                            @@@@   @@@@   &@@@@@@ ,%@       
                         @@@@   @@@,  /@@              @    
                        @@@   @@@@   @@@              @     
                      @@@@   @@@@   @@@                @  @ 
                      @@@   @@@@   ,@@@                @ @  
                     @@@@   @@@@    @@@@              @@ @ @
                     @@@@   @@@@@    @@@@@          @@@ @@ @
                     @@@@    @@@@@     @@@@@@@@@@@@@@  @@  @
                      @@@@    @@@@@@        @@@&     @@@  @ 
                      @@@@@     @@@@@@@@         %@@@@#  @@ 
                        @@@@#      @@@@@@@@@@@@@@@@@   @@   
                         &@@@@@          @@@@(       @@&    
                            @@@@@@@             /@

In [ ]:
# Run Example 3
%cd /home/scoped/work/example_3
! seisflows examples run 3 -r /home/scoped/specfem2d --with_mpi

In [ ]:
# Plots one synthetic waveform exported by the Solver
! seisflows plotst output/solver/001/syn/AA.S000000.BXY.semd --savefig AA.S000000.BXY.semd.png

# Plots multiple synthetic waveforms exported by the Solver
! seisflows plotst output/solver/001/syn/AA.S00000?.BXY.semd --savefig AA.S00000n.BXY.semd.png

Using the navigation menu on the left, navigate to:

`home -> work -> example_3`

and **open** the .png files that were created to look at the waveforms. The waveforms should be relatively simple since this is a homogeneous halfspace model.

## 7) Testing SPECFEM3D

We want to make sure we can run SPECFEM3D natively by running the homogeneous halfspace example problem using MPI.
First we need to set up the example directory, and then we'll run a 3D example problem.  
**Benchmark time for cell: ~9m**

In [ ]:
%cd /home/scoped/specfem3d/EXAMPLES/homogeneous_halfspace

! ln -s /home/scoped/specfem3d/bin .
! cp -r ./meshfem3D_files ./DATA/meshfem3D_files
! mkdir -p OUTPUT_FILES/DATABASES_MPI

In [ ]:
! mpirun -n 4 bin/xmeshfem3D
! mpirun -n 4 bin/xgenerate_databases
! mpirun -n 4 bin/xspecfem3D

Unfortunately we cannot conveniently plot the 3D model with Python. However we can use SeisFlows to plot the waveforms that were generated

In [ ]:
! seisflows plotst OUTPUT_FILES/*semd --savefig specfem3d_synthetics.png

Using the navigation menu on the left, navigate to:

`home -> specfem3d -> EXAMPLES -> homogeneous_halfspace`

and **open** the `specfem3d_synthetics.png` file to look at the waveforms. We can see the directivity of the moment tensor in the variation of amplitude of the given waveforms.

## 8) Testing PySEP/RecSec

Just want to make sure PySEP and RecSec work as RecSec will be used to plot SPECFEM2D synthetics during the workshop.

In [ ]:
%cd /home/scoped/work

# Download data for an example Anchorange event
! pysep -p mtuq_workshop_2022 -e 2009-04-07T201255_ANCHORAGE.yaml

In [ ]:
# Test RECord SECtion tool
! recsec --pysep_path 2009-04-07T201255_SOUTHERN_ALASKA/SAC \
    --min_period 10 --max_period 100 --move_out 4 --zero_pad_s 100 100 \
    --save 2009-04-07T201255_SOUTHERN_ALASKA/recsec_test.png

Using the navigation menu on the left, navigate to:

`home -> work -> 2009-04-07T201255_SOUTHERN_ALASKA`

This is the directory created by PySEP. Appropriately headed SAC files are downloaded to the directory SAC/. 

**Open** the `recsec_test.png` file to look at the created record section, and have a look in the `SAC/` directory to see all the waveforms downloaded.

### Testing RecSec with SPECFEM2D Synthetics

Finally, we can just make sure our record section works with synthetic waveforms.

>__NOTE:__ This requires having run the cell(s) in Section (1).

In [ ]:
# Try RecSec with SPECFEM2D synthetics
# The `--cartesian` flag tells RecSec that the SPECFEM2D domain is cartesian (not geographic)
%cd /home/scoped/specfem2d
! recsec --syn_path OUTPUT_FILES/ --cmtsolution DATA/SOURCE --stations DATA/STATIONS --cartesian

Using the navigation menu on the left, navigate to:

`home -> work -> specfem2d`

**Open** the `record_section.png` file to look at the created record section.

-------------

*All done. Thanks :)*